In [4]:
import time
load("kraus_tables.sage")

In [5]:
DB = CremonaDatabase(); DB
p = 3
kraus = kraus_table_Q3

In [6]:
# need to impose bad additive reduction and potentially good reduction (j valuation >= 0)
# then compute monodromy type and record Kodaira type

def mtype_ktype(conductor_range):
    t0 = time.time()    
    label_type_tuples = []
    no_curves = 0
    
    for E in DB.iter(conductor_range):
        no_curves += 1
        if valuation(E.j_invariant(), p) >= 0:
            # run Tate's algorithm to get reduction type
            LD = E.local_data(p)
            if LD.has_additive_reduction():
                # run Kraus's algorithm to determine monodromy type (returns group order)
                m_type = kraus(E)
                label_type_tuples.append((E.cremona_label(), 
                                          LD.kodaira_symbol(), 
                                          m_type))
    t1 = time.time()
    
    print('{} curves in conductor range.'.format(no_curves))
    print('{} curves with bad potentially good reduction.'.format(len(label_type_tuples)))
    print("Time: {}".format(t1-t0))
    return label_type_tuples

In [8]:
label_type_tuples = []

t0 = time.time()
for i in range(5): # adjust this as needed; set to 40 for full table, at present
    (N0, N1) = 10000*i, 10000*(i+1)
    conductor_range = iter(range(N0, N1, 2))
    print('Conductor range: {}'.format((N0, N1, p)))
    label_type_tuples += mtype_ktype(conductor_range)
    print
t1 = time.time()

print('TOTAL RUNTIME: {}'.format(t1-t0))  #

Conductor range: (0, 10000, 3)
50462 curves in conductor range.
7475 curves with bad potentially good reduction.
Time: 35.8027670383

Conductor range: (10000, 20000, 3)
53983 curves in conductor range.
8210 curves with bad potentially good reduction.
Time: 36.429915905

Conductor range: (20000, 30000, 3)
53554 curves in conductor range.
8580 curves with bad potentially good reduction.
Time: 36.9982590675

Conductor range: (30000, 40000, 3)
53121 curves in conductor range.
8085 curves with bad potentially good reduction.
Time: 35.634706974

Conductor range: (40000, 50000, 3)
53480 curves in conductor range.
8174 curves with bad potentially good reduction.
Time: 37.146697998

TOTAL RUNTIME: 182.016019106


In [67]:
objfile = os.path.join('./', '3-ktype_mtype_data.sobj')
save(label_type_tuples, objfile)

In [68]:
ktype_given_mtype = dict.fromkeys([2, 3, 4, 6, 8, 12, 24], [])
for (_, ktype, mtype) in label_type_tuples:
    if ktype not in ktype_given_mtype[mtype]:
        ktype_given_mtype[mtype] = ktype_given_mtype[mtype] + [ktype]
ktype_given_mtype

{2: [I0*],
 3: [II, II*],
 4: [III, III*],
 6: [IV, IV*],
 8: [],
 12: [IV*, II*, II, IV],
 24: []}

In [69]:
first_examples = []
for (label, ktype, mtype) in label_type_tuples:
    if (ktype, mtype) not in [e[1:] for e in first_examples]:
        first_examples += [(label, ktype, mtype)]

objfile = os.path.join('./', '3-ktype_mtype_first_examples.sobj')
save(first_examples, objfile)
first_examples = load(objfile)

In [70]:
first_examples

[('36a1', III, 4),
 ('36a3', III*, 4),
 ('54a1', IV*, 12),
 ('54a2', II*, 12),
 ('54a3', II, 12),
 ('54b3', IV, 12),
 ('126a1', I0*, 2),
 ('162a1', IV, 6),
 ('162a2', IV*, 6),
 ('162b1', II, 3),
 ('162b2', II*, 3)]

In [8]:
def k_type_to_cpt_group(symbol):
    if symbol in [KodairaSymbol('II'), KodairaSymbol('II*')]:
        return "0"
    elif symbol in [KodairaSymbol('III'), KodairaSymbol('III*')]:
        return "Z/2"
    elif symbol in [KodairaSymbol('IV'), KodairaSymbol('IV*')]:
        return "Z/3"
    elif str(symbol)[-1] == '*':
        if int(str(symbol)[1:-1]) % 2 == 0:
            return "Z/2 x Z/2"
        else:
            return "Z/4"
    else:
        print "Invalid reduction type"

In [9]:
for m_type in ktype_given_mtype:
    cpt_gps = map(lambda t: k_type_to_cpt_group(t), ktype_given_mtype[m_type])
    cpt_gps = list(set(cpt_gps))
    print(m_type, cpt_gps)

(2, ['0', 'Z/2 x Z/2'])
(3, ['Z/3'])
(4, ['Z/2'])
(6, ['Z/2 x Z/2', '0'])
(8, ['Z/2 x Z/2', 'Z/2', 'Z/4', '0'])
(24, ['0', 'Z/2', 'Z/4', 'Z/2 x Z/2'])


In [71]:
# remarks:
# McCallum proves exponent of inertial monodromy kills component group
# exponents: 2, 3, 4, 6, 8, 24 |-> 2, 3, 4, 6, 4, 12
#
# for C_2, the possible component group are: 0, C_2, C_2 x C_2,
#        but we don't see C_2
# note: in this case, monodromy group smaller than component group
# note: I think Dokchitser paper shows this is the only situation where potentially ordinary
#       reduction is possible.
#
# for C_3, possibilities are 0 and C_3, 
#        but we don't see 0
#
# for C_4, possibilities are 0, C_2, C_4, C_2 x C_2,
#        but we ONLY see one of these, C_2
#
# for C_6, possibilities are 0, C_2, C_3, C_2 x C_2,
#        but we don't see C_2 or C_3
# note: C_2 x C_2 is not a subquotient of C_6
# note: Dokchitsers show this is potentially supersingular
#
# for Q8, the possiblities are 0, C_2, C_4, C_2 x C_2, and we see all of these
# 
# for SL_2(F_3), the possiblities are 0, C_2, C_4, C_2 x C_2, and wee see all of these

In [73]:
factor(162)

2 * 3^4